In [1]:
from pathlib import Path

from aoc.decorators import timeit

data_file = Path("../Data/day9.txt").read_text()

EXAMPLE = "12345"
EXAMPLE2 = "2333133121414131402"


def prepare(input: str):
    return list(map(int, input.splitlines()[0]))

In [2]:
from copy import deepcopy
from dataclasses import dataclass
from typing import Iterable

RUN_EXAMPLES = True


@dataclass
class DiskItem:
    id: int
    blocks: int
    free_space: int


def make_disk_map(items: list[int]) -> list[DiskItem]:
    buffer: list[int] = []
    chunks: list[DiskItem] = []
    chunk_size = 2
    for i, item in enumerate(items):
        buffer.append(item)
        if (i % chunk_size) == (chunk_size - 1):
            chunks.append(DiskItem(id=len(chunks), blocks=buffer[0], free_space=item))
            buffer = []

    if len(buffer) > 0:
        chunks.append(DiskItem(id=len(chunks), blocks=buffer[0], free_space=0))

    return chunks


def join_string_list(string_list: Iterable[str]):
    return "".join(string_list)


def make_initial_visual_map(disk_map: list[DiskItem]):
    visual_map: list[str] = []
    for disk_item in disk_map:
        visual_map += ([str(disk_item.id)] * disk_item.blocks) + (
            ["."] * disk_item.free_space
        )

    return join_string_list(visual_map)


def compress_disk(
    visual_map: list[str], *, iterations: int | None = None, debug: bool = False
):
    new_visual_map = deepcopy(visual_map)
    if debug:
        print(join_string_list(new_visual_map))

    last_free_space_index = 0
    current_iteration = 0
    for i in reversed(range(len(new_visual_map))):
        if iterations is not None and current_iteration >= iterations:
            result = join_string_list(new_visual_map)
            if debug:
                # print("🐸 reached max iterations")
                print(result)

            return result

        current_item = new_visual_map[i]
        if current_item == ".":
            continue

        if i < last_free_space_index:
            result = join_string_list(new_visual_map)
            if debug:
                # print("🐸 caught up", i)
                print(result)

            return result

        map_item = new_visual_map[last_free_space_index]
        while map_item != ".":
            last_free_space_index += 1
            if last_free_space_index > i:
                result = join_string_list(new_visual_map)
                if debug:
                    print("🐸 no more free spots")
                    print(result)

                return result

            map_item = new_visual_map[last_free_space_index]

        assert map_item == "."

        current_iteration += 1
        new_visual_map[last_free_space_index], new_visual_map[i] = (
            current_item,
            map_item,
        )
        if debug:
            print(join_string_list(new_visual_map))

    result = join_string_list(new_visual_map)
    if debug:
        # print("🐸 ended with glory")
        print(result)

    return result


def make_checksum(compressed_disk: str, disk_map: list[DiskItem], *, debug: bool):
    def filter_item(item: str):
        if item == ".":
            return False

        return True

    def map_item(enumerated_item: tuple[int, str]):
        index, item = enumerated_item

        # disk_item = disk_map[int(item)]
        return index * int(item)

    products = list(map(map_item, enumerate(filter(filter_item, compressed_disk))))
    if debug:
        print(products)

    return sum(products)


if RUN_EXAMPLES:
    example2_visual_map = list(
        make_initial_visual_map(make_disk_map(prepare(EXAMPLE2)))
    )
    assert (
        compress_disk(example2_visual_map, iterations=0)
        == "00...111...2...333.44.5555.6666.777.888899"
    )
    assert (
        compress_disk(example2_visual_map, iterations=1)
        == "009..111...2...333.44.5555.6666.777.88889."
    )
    assert (
        compress_disk(example2_visual_map, iterations=2)
        == "0099.111...2...333.44.5555.6666.777.8888.."
    )
    assert (
        compress_disk(example2_visual_map, iterations=3)
        == "00998111...2...333.44.5555.6666.777.888..."
    )
    assert (
        compress_disk(example2_visual_map, iterations=4)
        == "009981118..2...333.44.5555.6666.777.88...."
    )
    assert (
        compress_disk(example2_visual_map, iterations=5)
        == "0099811188.2...333.44.5555.6666.777.8....."
    )
    assert (
        compress_disk(example2_visual_map, iterations=6)
        == "009981118882...333.44.5555.6666.777......."
    )
    assert (
        compress_disk(example2_visual_map, iterations=7)
        == "0099811188827..333.44.5555.6666.77........"
    )
    assert (
        compress_disk(example2_visual_map, iterations=8)
        == "00998111888277.333.44.5555.6666.7........."
    )
    assert (
        compress_disk(example2_visual_map, iterations=9)
        == "009981118882777333.44.5555.6666..........."
    )
    assert (
        compress_disk(example2_visual_map, iterations=10)
        == "009981118882777333644.5555.666............"
    )
    assert (
        compress_disk(example2_visual_map, iterations=11)
        == "00998111888277733364465555.66............."
    )
    final_compress_disk_result = compress_disk(example2_visual_map, iterations=12)
    assert final_compress_disk_result == "0099811188827773336446555566.............."
    assert final_compress_disk_result == compress_disk(example2_visual_map.copy())


@timeit
def part1(input: str, *, debug=False):
    disk_map = make_disk_map(prepare(input))
    visual_map = list(make_initial_visual_map(disk_map))
    compressed_disk = compress_disk(deepcopy(visual_map), debug=debug)

    return make_checksum(compressed_disk, disk_map, debug=debug)


if RUN_EXAMPLES:
    example_result = part1(EXAMPLE2, debug=True)

    assert (
        example_result == 1928
    ), f"Expected example result to be 1928, but got {example_result} instead"

result = part1(data_file)

print("result is", result)

assert (
    result > 90575306662
), f"Expected result to be greater than 90575306662, but got {result} instead"

00...111...2...333.44.5555.6666.777.888899
009..111...2...333.44.5555.6666.777.88889.
0099.111...2...333.44.5555.6666.777.8888..
00998111...2...333.44.5555.6666.777.888...
009981118..2...333.44.5555.6666.777.88....
0099811188.2...333.44.5555.6666.777.8.....
009981118882...333.44.5555.6666.777.......
0099811188827..333.44.5555.6666.77........
00998111888277.333.44.5555.6666.7.........
009981118882777333.44.5555.6666...........
009981118882777333644.5555.666............
00998111888277733364465555.66.............
0099811188827773336446555566..............
🐸 no more free spots
0099811188827773336446555566..............
[0, 0, 18, 27, 32, 5, 6, 7, 64, 72, 80, 22, 84, 91, 98, 45, 48, 51, 108, 76, 80, 126, 110, 115, 120, 125, 156, 162]
def part1(input, debug): took: 0.0002 sec
def part1(input, debug): took: 0.1115 sec
result is 90575306662


AssertionError: Expected result to be greater than 90575306662, but got 90575306662 instead

In [ ]:
@timeit
def part2(input: str):
    return 0


example_result = part2(EXAMPLE)

assert (
    example_result == 0
), f"Expected example result to be 0, but got {example_result} instead"

result = part2(data_file)

print("result is", result)

assert result == 0